In [1]:
# For data processing
import numpy as np
from math import sqrt
# For data processing and manipulation
import pandas as pd
import csv
 
# For date calculations
import datetime
import time
 
# For ploting data
import IPython
import IPython.display
 
import itertools
from itertools import cycle
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
 
# For checking path
import os , gc
from os import path
import csv
import json
from IPython.display import clear_output 
 
from scipy.stats import hmean
 
from sklearn.metrics import mean_squared_error
import requests

import ipywidgets as widgets
from ipywidgets import interact, interact_manual


In [2]:
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

In [3]:
DATABASE_URL = "http://127.0.0.1:5000/"

In [4]:
ROOTPATH = 'C:\\Study\\CSE 4000\\New folder\\Thesis\\'
#PATH_IMAGE = ROOTPATH+'images_chicago\\'
#PATH_WEAT_CHI = ROOTPATH+"dataset\\processed_weather_data_chicago_2004_2017.csv"

test query

In [5]:
requestData = {
    'wx_phrase': 'Mostly Cloudy',
    'pressure': 1003.99}
urlTest = DATABASE_URL + "/query/find/chicago/weather"
response = requests.get(urlTest, json=requestData)
if response.ok:
    try:
        data = response.json()
        print(len(data))
    except json.JSONDecodeError as e:
        print('Error in decoding json: ' + e)
    else:  
        print("Error:", response.status_code)

28
Error: 200


<h3><b> Phrase Types without Pipeline </h3></b>

In [44]:
# Record the start time
start_time = time.time()

requestData = {
 'field': 'wx_phrase'}

urlTest = DATABASE_URL + "/query/unique/chicago/weather"
x = requests.get(urlTest, json=requestData)
num_phrase = x.json()
print("Num Phrase :" , len(num_phrase))
for i in num_phrase:
    print(i)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time
print("Execution Time: {:.2f} seconds".format(execution_time))

Num Phrase : 63

Blowing Snow
Blowing Snow / Windy
Cloudy
Cloudy / Windy
Drizzle
Drizzle and Fog
Fair
Fair / Windy
Fog
Fog / Windy
Funnel Cloud
Haze
Haze / Windy
Heavy Rain
Heavy Rain / Windy
Heavy Snow
Heavy Snow / Windy
Heavy T-Storm
Heavy T-Storm / Windy
Light Drizzle
Light Drizzle / Windy
Light Freezing Drizzle
Light Freezing Rain
Light Rain
Light Rain / Windy
Light Rain with Thunder
Light Sleet
Light Sleet / Windy
Light Snow
Light Snow / Freezing Rain
Light Snow / Windy
Light Snow and Sleet
Light Snow and Sleet / Windy
Mostly Cloudy
Mostly Cloudy / Windy
Partly Cloudy
Partly Cloudy / Windy
Patches of Fog
Rain
Rain / Freezing Rain
Rain / Windy
Rain and Sleet
Rain and Snow
Rain and Snow / Windy
Shallow Fog
Sleet / Windy
Smoke
Snow
Snow / Freezing Rain
Snow / Windy
Snow and Sleet
Snow and Sleet / Windy
Snow and Thunder
T-Storm
T-Storm / Windy
Thunder
Thunder / Windy
Thunder in the Vicinity
Tornado
Widespread Dust / Windy
Wintry Mix
Wintry Mix / Windy
Execution Time: 0.18 seconds


In [45]:
del x

<h3><b> Phrase Types using Pipeline </h3></b>

In [6]:
# Record the start time
start_time = time.time()

phraseXCount = {}
urlTest = DATABASE_URL + "/query/aggregate/chicago/weather"
pipeline = [
    {
        "$group": {
                "_id": "$wx_phrase",
                "Count" : {"$sum" : 1}
        }
    }
]
x = requests.get(urlTest, json=pipeline)
data = x.json()
tsum = sum([x['Count'] for x in data])
c = 0
for doc in data:
    print(doc)
    phraseXCount[doc['_id']] = doc['Count']
    c = c + 1

# Record the end time
end_time = time.time()
execution_time = end_time - start_time


print("Total Unique Types = " , c)
print("Total Cases = " , tsum)
print("Execution Time: {:.2f} seconds".format(execution_time))
        

{'Count': 379, '_id': 'Thunder'}
{'Count': 21, '_id': 'Smoke'}
{'Count': 225, '_id': 'Haze'}
{'Count': 2, '_id': 'Funnel Cloud'}
{'Count': 56, '_id': 'Thunder in the Vicinity'}
{'Count': 1, '_id': 'Light Snow / Freezing Rain'}
{'Count': 1, '_id': 'Light Sleet / Windy'}
{'Count': 4, '_id': 'Light Sleet'}
{'Count': 3, '_id': 'Shallow Fog'}
{'Count': 42194, '_id': 'Fair'}
{'Count': 11, '_id': 'Blowing Snow'}
{'Count': 4918, '_id': 'Light Snow'}
{'Count': 1, '_id': 'Sleet / Windy'}
{'Count': 67, '_id': 'Heavy T-Storm / Windy'}
{'Count': 1, '_id': 'Widespread Dust / Windy'}
{'Count': 241, '_id': 'Heavy T-Storm'}
{'Count': 16, '_id': 'Heavy Rain / Windy'}
{'Count': 6, '_id': 'None'}
{'Count': 32, '_id': 'Light Freezing Drizzle'}
{'Count': 41577, '_id': 'Mostly Cloudy'}
{'Count': 34, '_id': 'Blowing Snow / Windy'}
{'Count': 362, '_id': 'Light Rain / Windy'}
{'Count': 39, '_id': 'T-Storm / Windy'}
{'Count': 507, '_id': 'Light Snow / Windy'}
{'Count': 5, '_id': 'Snow / Freezing Rain'}
{'Count':

In [47]:
del x

<h3><b> not working Average Temprature</h3></b>

In [7]:
# Record the start time
start_time = time.time()

urlTest = DATABASE_URL + "/query/aggregate/chicago/weather"
"""pipeline = [
    {
        "$group": {
            "_id": "$temp",
            "total_temp": {"$sum": 1},
        }
    },
    {
        "$group": {
            "_id": None,
            "average_temp_per_year": {"$avg": "$total_temp"},
        }
    }
]"""
pipeline = [
    {
        "$group": {
            "_id": None,
            "avg_temperature": { "$avg": "$temp" }
        }
    }
]
x = requests.get(urlTest, json=pipeline)
data = x.json()


# Print the result
for doc in data:
    print(doc)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time
print("Execution Time: {:.2f} seconds".format(execution_time))
        

{'_id': 'None', 'avg_temperature': 10.897020225413}
Execution Time: 0.08 seconds


In [49]:
del x

<h3><b> Max Temprature </h3></b>

In [50]:
# Record the start time
start_time = time.time()

urlTest = DATABASE_URL + "/query/aggregate/chicago/weather"
pipeline = [
    {
        "$group": {
            "_id": None,
            "max_temperature": { "$max": "$temp" }
        }
    }
]

x = requests.get(urlTest, json=pipeline)
data = x.json()

# Print the result
for doc in data:
    print(doc)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time
print("Execution Time: {:.2f} seconds".format(execution_time))
        

{'_id': 'None', 'max_temperature': 40.0}
Execution Time: 0.13 seconds


In [51]:
del x

<h3><b> nan asche min Temprature </h3></b>

In [52]:
# Record the start time
start_time = time.time()

urlTest = DATABASE_URL + "/query/aggregate/chicago/weather"
pipeline = [
    {
        "$group": {
            "_id": None,
            "min_temperature": { "$min": "$temp" }
        }
    }
]

x = requests.get(urlTest, json=pipeline)
data = x.json()

# Print the result
for doc in data:
    print(doc)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time
print("Execution Time: {:.2f} seconds".format(execution_time))
        

{'_id': 'None', 'min_temperature': nan}
Execution Time: 0.22 seconds


<h3><b> not working max temp per year </h3></b>

In [56]:
# Record the start time
start_time = time.time()

urlTest = DATABASE_URL + "/query/aggregate/chicago/weather"
pipeline = [
    {
        "$addFields": {
            "dateComponents": {
                "$split": [
                    {"$arrayElemAt": [{"$split": ["$valid_date_time", " "]}, 0]}, "/"
                ]
            }
        }
    },
    {
        "$addFields": {
            "date": {
                "$dateFromParts": {
                    "year": {"$toInt": {"$arrayElemAt": ["$dateComponents", 2]}} ,
                    "month": {"$toInt": {"$arrayElemAt": ["$dateComponents", 0]}},
                }
            }
        }
    },
    {
        "$group": {
            "_id": {
                "month": {"$month": "$date"}
            },
            "max_temp": {"$max": "$temp"}
        }
    },
    {
        "$project": {
            "_id": 0,
            "month": "$_id.month",
            "max_temp": 1
        }
    }
]
x = requests.get(urlTest, json=pipeline)
data = x.json()

# Print the result
for doc in data:
    print(doc)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time
print("Execution Time: {:.2f} seconds".format(execution_time))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

<h1><b>Count Rainy Days </h1></b>

In [63]:
# Record the start time
start_time = time.time()
import re
match_word = "Rain"
rainXcount = {}

urlTest = DATABASE_URL + "/query/aggregate/chicago/weather"

pipeline = [
    {
        "$match": {
            "wx_phrase": {"$regex": match_word, "$options": "i"}
        }
    }
]

x = requests.get(urlTest, json=pipeline)
data = x.json()

c = 0
# Print the result
for doc in data:
    rainXcount[doc['_id']] = doc['expire_date_time']
    print(f"On date : {doc['expire_date_time']} , {doc['wx_phrase']} was observed")
    c = c + 1
print("Total Cases = " , c)

# Record the end time
end_time = time.time()
execution_time = end_time - start_time
print("Execution Time: {:.2f} seconds".format(execution_time))

On date : 19/03/2014 17:00:00 , Light Rain was observed
On date : 19/03/2014 18:00:00 , Light Rain was observed
On date : 19/03/2014 18:00:00 , Light Rain was observed
On date : 19/03/2014 19:00:00 , Light Rain was observed
On date : 19/03/2014 20:00:00 , Rain was observed
On date : 19/03/2014 20:00:00 , Light Rain was observed
On date : 19/03/2014 20:00:00 , Light Rain was observed
On date : 19/03/2014 21:00:00 , Light Rain was observed
On date : 19/03/2014 22:00:00 , Light Rain was observed
On date : 19/03/2014 23:00:00 , Light Rain was observed
On date : 20/03/2014 00:00:00 , Light Rain was observed
On date : 22/03/2014 09:00:00 , Light Rain was observed
On date : 22/03/2014 10:00:00 , Light Rain was observed
On date : 22/03/2014 11:00:00 , Light Rain was observed
On date : 27/03/2014 17:00:00 , Light Rain was observed
On date : 27/03/2014 18:00:00 , Light Rain was observed
On date : 27/03/2014 18:00:00 , Light Rain was observed
On date : 27/03/2014 19:00:00 , Light Rain was observe

In [58]:
del x

<h1><b>Wind Direction Distribution </h1></b>


In [72]:
# Record the start time
start_time = time.time()

#Arrest count
wdirXCount = {}
urlTest = DATABASE_URL + "/query/aggregate/chicago/weather"
pipeline = [
    {
        "$group": {
                "_id": "$wdir",
                "Intensity" : {"$sum" : 1}
        }
    },
    {
        "$sort": {
            "_id": 1
        }
    }
]
x = requests.get(urlTest, json=pipeline)
data = x.json()
tsum = sum([x['Intensity'] for x in data])
c = 0
for doc in data:
    print(doc)
    wdirXCount[doc['_id']] = doc['Intensity']
    c = c + 1

# Record the end time
end_time = time.time()
execution_time = end_time - start_time

print("Total directions = " , c)
print("Total Cases = " , tsum)
print("Execution Time: {:.2f} seconds".format(execution_time))
        

{'Intensity': 12837, '_id': 'nan'}
{'Intensity': 4672, '_id': '10.0'}
{'Intensity': 5010, '_id': '20.0'}
{'Intensity': 5464, '_id': '30.0'}
{'Intensity': 4193, '_id': '40.0'}
{'Intensity': 3804, '_id': '50.0'}
{'Intensity': 4277, '_id': '60.0'}
{'Intensity': 4188, '_id': '70.0'}
{'Intensity': 4648, '_id': '80.0'}
{'Intensity': 4798, '_id': '90.0'}
{'Intensity': 4307, '_id': '100.0'}
{'Intensity': 3142, '_id': '110.0'}
{'Intensity': 2851, '_id': '120.0'}
{'Intensity': 2748, '_id': '130.0'}
{'Intensity': 3199, '_id': '140.0'}
{'Intensity': 2868, '_id': '150.0'}
{'Intensity': 3678, '_id': '160.0'}
{'Intensity': 5155, '_id': '170.0'}
{'Intensity': 6148, '_id': '180.0'}
{'Intensity': 7328, '_id': '190.0'}
{'Intensity': 9216, '_id': '200.0'}
{'Intensity': 8795, '_id': '210.0'}
{'Intensity': 6438, '_id': '220.0'}
{'Intensity': 5725, '_id': '230.0'}
{'Intensity': 6660, '_id': '240.0'}
{'Intensity': 6774, '_id': '250.0'}
{'Intensity': 6079, '_id': '260.0'}
{'Intensity': 6832, '_id': '270.0'}
{'

In [67]:
del x

<h1><b>In A Range Check</h1></b>


In [79]:
# Record the start time
start_time = time.time()

urlTest = DATABASE_URL + "/query/aggregate/chicago/weather"
pipeline = [
    {
        "$match": {
            "$and": [
                {"pressure": {"$lt": 1001.00}},
                {"pressure": {"$gt": 1000.00}}
            ]
        }
    }
]
x = requests.get(urlTest, json=pipeline)
data = x.json()
#tsum = sum([x['pressure_days'] for x in data])
c = 0
for doc in data:
    print(doc)
    #arrestXCount[doc['_id']] = doc['Number of Cases']
    c = c + 1

# Record the end time
end_time = time.time()
execution_time = end_time - start_time


print("Total Cases= " , c)
print("Execution Time: {:.2f} seconds".format(execution_time))
        

{'': 11, '_id': '652c1ed757fa4f5aaad90323', 'expire_date_time': '02/01/2004 01:00:00', 'expire_time_gmt': 1072983180, 'heat_index': 3.0, 'max_temp': nan, 'min_temp': nan, 'pressure': 1000.08, 'rh': 65.0, 'temp': 3.0, 'valid_date_time': '01/01/2004 23:00:00', 'valid_time_gmt': 1072975980, 'vis': 16.0, 'wc': -1.0, 'wdir': 170.0, 'wspd': 19.0, 'wx_phrase': 'Cloudy'}
{'': 147, '_id': '652c1ed757fa4f5aaad903ab', 'expire_date_time': '06/01/2004 00:00:00', 'expire_time_gmt': 1073325180, 'heat_index': -5.0, 'max_temp': nan, 'min_temp': nan, 'pressure': 1000.96, 'rh': 81.0, 'temp': -5.0, 'valid_date_time': '05/01/2004 22:00:00', 'valid_time_gmt': 1073317980, 'vis': 16.0, 'wc': -12.0, 'wdir': 330.0, 'wspd': 24.0, 'wx_phrase': 'Mostly Cloudy'}
{'': 149, '_id': '652c1ed757fa4f5aaad903ad', 'expire_date_time': '06/01/2004 02:00:00', 'expire_time_gmt': 1073332380, 'heat_index': -4.0, 'max_temp': -3.0, 'min_temp': -5.0, 'pressure': 1000.76, 'rh': 78.0, 'temp': -4.0, 'valid_date_time': '06/01/2004 00:0

In [74]:
del x